In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

# 0. Load Data

In [2]:
X_train = pd.read_csv('../data/X_train.csv')
y_train = pd.read_csv('../data/y_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
X_train.head()

,물품_카테고리,new_송하인grid,new_수하인grid
0,4.092014,4.609900,5.483871
1,4.976169,5.396163,6.012577
2,4.322081,4.609900,4.264901
3,6.088825,4.972973,6.012577
4,4.322081,4.609900,4.214660


# 1. 교차검증

## 1) Train-Test Split

        학습용 데이터를 8:2의 비율로 학습용:검증용 데이터로 재분포한다.

        sklearn 내부에 train_test_split으로 분리할 수 있다.

In [3]:
from sklearn.model_selection import train_test_split

- 데이터를 바로 분리할 수 있다.

In [4]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=0) # 데이터를 각각 쪼개는 방법입니다.

# X와 y는 서로 매칭되어야하기 때문에, 바로 분리할 때는 이렇게 데이터를 함께 넣어주어야 합니다.
# X_train을 X_tr, X_val로 8:2의 비율로 분해하고
# y_train도 y_tr, y_val로 8:2의 비율로 분해하며
# 데이터는 shuffle 한 상태로 받아옵니다.


print('Train Data : ', X_tr.shape, y_tr.shape)
print('Valid Data : ', X_val.shape, y_val.shape)

Train Data :  (25347, 3) (25347, 1)
Valid Data :  (6337, 3) (6337, 1)


- Index를 사용하여 분리할 수 있다.(권장)

In [5]:
train_idx, valid_idx = train_test_split(X_train.index, test_size=0.2, shuffle=True, random_state=0)

# 위에서 데이터를 바로 분리해주었다면, 여기서는 indexing을 이용하여 데이터를 분리합니다.
# 이 방법이 앞으로 더 많이 사용되니 참고해주시면 됩니다.

# X_train과 y_train을 직접 데이터를 매칭하기 위해 데이터안에 모두 넣어주지 않고, index번호를 활용해 매칭합니다.
# 나중에는 X_train이 여러개가 될 수 있으며, 이미지와 같은 데이터는 번호를 활용해야하기에 indexing을 권장드립니다.

X_tr = X_train.iloc[train_idx]; y_tr = y_train.iloc[train_idx]
X_val = X_train.iloc[valid_idx]; y_val = y_train.iloc[valid_idx]

print('Train Data : ', X_tr.shape, y_tr.shape)
print('Valid Data : ', X_val.shape, y_val.shape)

Train Data :  (25347, 3) (25347, 1)
Valid Data :  (6337, 3) (6337, 1)


## 2) K-Fold

        데이터의 낭비를 막고, 좀 더 일반화하된 결과를 얻기 위해 K-FOld를 사용한다.
        
        sklearn에 KFold를 이용해 사용할 수 있다.

        이 경우, index를 이용한 데이터 분리가 적절하다.

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0) # 데이터를 어떻게 split할지 방법을 적어줍니다. 데이터를 5등분하고, shuffle = True로 지정합니다.

for num, (train_idx, valid_idx) in enumerate(kf.split(X_train.index)) :  # X_train의 index를 이용해 데이터를 쪼갤것입니다.
    print(f'=== Fold {num} ===')
    X_tr = X_train.iloc[train_idx]; y_tr = y_train.iloc[train_idx]  # indexing을 이용해 데이터를 쪼갭니다.
    X_val = X_train.iloc[valid_idx]; y_val = y_train.iloc[valid_idx]

    print('\tTrain Data : ', X_tr.shape, y_tr.shape)
    print('\tValid Data : ', X_val.shape, y_val.shape)

=== Fold 0 ===
	Train Data :  (25347, 3) (25347, 1)
	Valid Data :  (6337, 3) (6337, 1)
=== Fold 1 ===
	Train Data :  (25347, 3) (25347, 1)
	Valid Data :  (6337, 3) (6337, 1)
=== Fold 2 ===
	Train Data :  (25347, 3) (25347, 1)
	Valid Data :  (6337, 3) (6337, 1)
=== Fold 3 ===
	Train Data :  (25347, 3) (25347, 1)
	Valid Data :  (6337, 3) (6337, 1)
=== Fold 4 ===
	Train Data :  (25348, 3) (25348, 1)
	Valid Data :  (6336, 3) (6336, 1)


## 3) 교차검증을 활용한 모델 성능 예측

        지난 시간에 배웠던 "성과지표"의 수업을 평가하는 방법이다.

        sklearn에 mean_squared_error를 이용해 확인할 수 있다.

        mean_squared_error는 MSE로, mean_squared_error(예측값, 실제값) 으로 성능을 확인할 수 있다.

        우리가 평가지표로 사용하는 RMSE는 위의 MSE에 root를 씌운 값으로,

        np.sqrt()를 활용해 RMSE를 구현할 수 있다.

- 모델 정의

In [7]:
linr = LinearRegression()

- train_test_split

In [8]:
from sklearn.metrics import mean_squared_error

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=0) # Data Split

linr.fit(X_tr, y_tr) # Model Fitting 
val_pred = linr.predict(X_val) # Predict Validation Data

mse_score = mean_squared_error(val_pred, y_val) # Check MSE Score
rmse_score = np.sqrt(mse_score) # Make MSE Score to RMSE Score with np.sqrt() 

print('Validation Score : ', rmse_score)


Validation Score :  6.4475356652653


- k-fold

In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

for num, (train_idx, valid_idx) in enumerate(kf.split(X_train.index)) : 
    print(f'=== Fold {num} ===')
    X_tr = X_train.iloc[train_idx]; y_tr = y_train.iloc[train_idx] # index 번호를 이용하여 데이터 split
    X_val = X_train.iloc[valid_idx]; y_val = y_train.iloc[valid_idx]

    linr.fit(X_tr, y_tr)
    val_pred = linr.predict(X_val)

    mse_score = mean_squared_error(val_pred, y_val)
    rmse_score = np.sqrt(mse_score)

    print('\tValidation Score : ', rmse_score)

=== Fold 0 ===
	Validation Score :  6.447535665265286
=== Fold 1 ===
	Validation Score :  8.615181795533024
=== Fold 2 ===
	Validation Score :  5.304771633092518
=== Fold 3 ===
	Validation Score :  5.046098460380502
=== Fold 4 ===
	Validation Score :  6.103155030570359


- 점수만 확인하고 싶을 때는, cross_val_score를 사용합니다.

        cross_val_score는 교차검증 점수를 반환해주는 방법입니다.

        모델, 학습용 데이터, 목표 변수, 교차검증에 사용할 방법, 점수방법을 설정하면, 점수를 반환합니다.

        아래의 "neg_root_mean_squared_error'는 RMSE를 음수로 측정한 값입니다.

        따라서 abs(점수)를 활용해 절대값을 씌워줍니다.

In [10]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(estimator = linr,  # 측정할 모델
                        X = X_train,  # 학습용 데이터
                        y = y_train,  # Target 데이터
                        cv = kf,  # Cross-Validation = K-Fold로 지정, 다른 cv를 지정해도 됩니다.
                        scoring = 'neg_root_mean_squared_error') # 평가지표는 RMSE로 지정합니다.
score = abs(score)
print('Total Validation Score : ', abs(score))
print('Mean of Validation Score : ', np.mean(score))
print('Minimum of Validation Score : ', min(score))
print('Maximum of Validation Score : ', max(score))

Total Validation Score :  [6.44753567 8.6151818  5.30477163 5.04609846 6.10315503]
Mean of Validation Score :  6.303348516968338
Minimum of Validation Score :  5.046098460380502
Maximum of Validation Score :  8.615181795533024


- k-fold를 적용하여 예측하기

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

test_preds = []

for num, (train_idx, valid_idx) in enumerate(kf.split(X_train.index)) : 
    print(f'=== Fold {num} ===')
    X_tr = X_train.iloc[train_idx]; y_tr = y_train.iloc[train_idx]
    X_val = X_train.iloc[valid_idx]; y_val = y_train.iloc[valid_idx]

    linr.fit(X_tr, y_tr)
    val_pred = linr.predict(X_val)
    test_pred = linr.predict(X_test)
    test_preds.append(test_pred)

    mse_score = mean_squared_error(val_pred, y_val)
    rmse_score = np.sqrt(mse_score)

    print('\tValidation Score : ', rmse_score)

=== Fold 0 ===
	Validation Score :  6.447535665265286
=== Fold 1 ===
	Validation Score :  8.615181795533024
=== Fold 2 ===
	Validation Score :  5.304771633092518
=== Fold 3 ===
	Validation Score :  5.046098460380502
=== Fold 4 ===
	Validation Score :  6.103155030570359


In [12]:
test_pred = np.mean(test_preds, axis= 0)

sample = pd.read_csv('../data/sample_submission.csv')
sample['운송장_건수'] =test_pred
sample.to_csv('../submissions/교차검증.csv',index=False)

sample.head()

,index,운송장_건수
0,0,17.194486
1,1,9.064716
2,2,6.720405
3,3,4.965252
4,4,5.457053
